In [1]:
!pip install -qqU --progress-bar off datasets

In [2]:
from kaggle_secrets import UserSecretsClient
import datasets as hfds
import huggingface_hub
import numpy as np
hfds.disable_caching()

In [3]:
!mkdir chunks
!ln -s /kaggle/input/*/*/* chunks

In [4]:
ds = [hfds.load_from_disk(f"chunks/chunk_{i}") for i in range(274)]
ds = [hfds.concatenate_datasets(list(d.values())) for d in ds]
assert len(ds) == 274
assert sum(map(len, ds)) == 60638126
ds = hfds.concatenate_datasets(ds)
assert len(ds) == 60638126, len(ds)
ds

Dataset({
    features: ['logits', 'mfcc'],
    num_rows: 60638126
})

In [5]:
ds = [ds.filter(lambda d: np.argmax(d['logits'], axis=-1) == c, batched=True) for c in range(12)]
assert len(ds) == 12
assert sum(map(len, ds)) == 60638126
assert min(map(len, ds)) == 171416
ds

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60638126 [00:00<?, ? examples/s]

[Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 3597855
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 394449
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 858498
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 564780
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 171416
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 205550
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 649916
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 1275944
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 926698
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 282037
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 38202935
 }),
 Dataset({
     features: ['logits', 'mfcc'],
     num_rows: 13508048
 })]

In [6]:
ds = [d.shuffle().select(range(min(map(len, ds)))) for d in ds]
assert len(ds) == 12
assert set(map(len, ds)) == {171416}
ds = hfds.concatenate_datasets(ds)
assert len(ds) == 12 * 171416, len(ds)
ds

Dataset({
    features: ['logits', 'mfcc'],
    num_rows: 2056992
})

In [7]:
hfds.disable_progress_bar()
huggingface_hub.utils.disable_progress_bars()

In [8]:
ds.push_to_hub('mazkooleg/0-9up_ft_ensemble_distilled_from_cv12_balanced_mfcc',
               token=UserSecretsClient().get_secret("hf-datasets-token"))

In [9]:
!echo RG9uZSAhISEK | base64 -d

Done !!!
